<a href="https://colab.research.google.com/github/sanghyun-ai/ktcloud_genai/blob/main/%EC%8B%A4%EC%8A%B5%EC%BD%94%EB%93%9C/106_LLM_(%EB%B3%B4%EC%B6%A9)_RNN_Attention_Transformer_%EA%B8%B0%EB%B3%B8%EA%B5%AC%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RNN vs Seq2Seq vs Attention vs Transformer**



---



- **장거리 의존성 문제(long-term dependency problem)**와
- **병렬 처리의 한계를 해결하는 방향으로 발전**함

## **1. RNN(Recurrent Neural Network)**


In [1]:
import torch
import torch.nn as nn

class SimpleRNN(nn.Module):
    """기본 RNN을 이용한 번역 모델 (교육용 간소화 버전)"""

    def __init__(self, input_vocab_size, output_vocab_size, hidden_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size

        # 임베딩 레이어
        self.embedding = nn.Embedding(input_vocab_size, hidden_size)

        # RNN 레이어 (순차적 처리)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)

        # 출력 레이어
        self.fc = nn.Linear(hidden_size, output_vocab_size)

    def forward(self, x):
        # x shape: (batch_size, sequence_length)

        # 임베딩: (batch_size, seq_len, hidden_size)
        embedded = self.embedding(x)

        # RNN 처리 (순차적으로 한 단어씩 처리)
        # output: (batch_size, seq_len, hidden_size)
        # hidden: (1, batch_size, hidden_size) - 마지막 hidden state
        output, hidden = self.rnn(embedded)

        # 각 시점의 출력을 단어 확률로 변환
        predictions = self.fc(output)

        return predictions, hidden

# 사용 예시
input_vocab_size = 1000   # 한국어 단어 수
output_vocab_size = 1000  # 영어 단어 수
hidden_size = 256

model = SimpleRNN(input_vocab_size, output_vocab_size, hidden_size)

# 예시 입력: "나는 점심 식사로 파스타를 먹을 예정입니다" (토큰화된 인덱스)
sample_input = torch.tensor([[10, 25, 43, 87, 102, 156]])  # shape: (1, 6)

output, hidden = model(sample_input)
print(f"RNN 출력 shape: {output.shape}")  # (1, 6, 1000)
print(f"최종 hidden state shape: {hidden.shape}")  # (1, 1, 256)
print("문제점: 긴 문장에서 '나는'의 정보가 마지막까지 전달되기 어려움!")

RNN 출력 shape: torch.Size([1, 6, 1000])
최종 hidden state shape: torch.Size([1, 1, 256])
문제점: 긴 문장에서 '나는'의 정보가 마지막까지 전달되기 어려움!


### 예제 1: Hidden State의 실제 값 확인하기

In [2]:
import torch
import torch.nn as nn

# 간단한 RNN 모델
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)

    def forward(self, x):
        # x: (batch_size, sequence_length, input_size)
        # output: 모든 시점의 hidden state
        # hidden: 마지막 hidden state
        output, hidden = self.rnn(x)
        return output, hidden

# 모델 생성
input_size = 10    # 단어 임베딩 차원
hidden_size = 20   # hidden state 차원
model = SimpleRNN(input_size, hidden_size)

# 예시 입력: 6개 단어 ("나는", "점심", "식사로", "파스타를", "먹을", "예정입니다")
# 실제로는 단어를 벡터로 변환한 값
sequence_length = 6
batch_size = 1
x = torch.randn(batch_size, sequence_length, input_size)

# Forward pass
all_hidden_states, final_hidden = model(x)

print("=" * 60)
print("각 시점의 Hidden State 확인")
print("=" * 60)

for t in range(sequence_length):
    words = ["나는", "점심", "식사로", "파스타를", "먹을", "예정입니다"]
    print(f"\nh{t+1} ('{words[t]}' 처리 후):")
    print(f"  Shape: {all_hidden_states[0, t].shape}")  # (hidden_size,)
    print(f"  값 샘플 (처음 5개): {all_hidden_states[0, t, :5].detach().numpy()}")
    print(f"  평균: {all_hidden_states[0, t].mean().item():.4f}")
    print(f"  표준편차: {all_hidden_states[0, t].std().item():.4f}")

print("\n" + "=" * 60)
print("최종 Hidden State (h6):")
print("=" * 60)
print(f"Shape: {final_hidden.shape}")  # (1, batch_size, hidden_size)
print(f"이것이 전체 문장의 의미를 압축한 벡터입니다!")

각 시점의 Hidden State 확인

h1 ('나는' 처리 후):
  Shape: torch.Size([20])
  값 샘플 (처음 5개): [ 0.29985452 -0.2139289  -0.29840955 -0.04969395 -0.67423046]
  평균: 0.0983
  표준편차: 0.4239

h2 ('점심' 처리 후):
  Shape: torch.Size([20])
  값 샘플 (처음 5개): [ 0.09721795 -0.07027858 -0.3024618   0.06140145  0.04500112]
  평균: -0.0653
  표준편차: 0.3114

h3 ('식사로' 처리 후):
  Shape: torch.Size([20])
  값 샘플 (처음 5개): [-0.29584005  0.23863187  0.31218764 -0.49818477  0.56312275]
  평균: -0.0041
  표준편차: 0.4587

h4 ('파스타를' 처리 후):
  Shape: torch.Size([20])
  값 샘플 (처음 5개): [-0.319365    0.59877855 -0.7083339  -0.49061483  0.03293834]
  평균: 0.0104
  표준편차: 0.4240

h5 ('먹을' 처리 후):
  Shape: torch.Size([20])
  값 샘플 (처음 5개): [ 0.45612144 -0.30095637 -0.48437113 -0.0419737  -0.05000209]
  평균: 0.0824
  표준편차: 0.3501

h6 ('예정입니다' 처리 후):
  Shape: torch.Size([20])
  값 샘플 (처음 5개): [ 0.19061457  0.06515525 -0.1807474  -0.34183508 -0.0019587 ]
  평균: -0.0002
  표준편차: 0.4072

최종 Hidden State (h6):
Shape: torch.Size([1, 1, 20])
이것이 전체 문장의 의미를 압축한 벡터입

### 예제 2: Hidden State가 어떻게 변화하는지 시각화

In [3]:
import torch
import torch.nn as nn
import numpy as np

# 더 상세한 RNN 클래스
class RNNWithHiddenTracking(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(RNNWithHiddenTracking, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)
        self.hidden_size = hidden_size

    def forward(self, x):
        embedded = self.embedding(x)

        # 각 시점의 hidden state를 수동으로 계산하여 추적
        batch_size = x.shape[0]
        seq_len = x.shape[1]

        # 초기 hidden state (h0) - 보통 0으로 초기화
        h = torch.zeros(1, batch_size, self.hidden_size)

        hidden_states = []

        for t in range(seq_len):
            # 한 단어씩 처리
            x_t = embedded[:, t:t+1, :]  # (batch_size, 1, embedding_dim)
            out, h = self.rnn(x_t, h)
            hidden_states.append(h.squeeze(0))  # (batch_size, hidden_size)

        return torch.stack(hidden_states, dim=1), h

# 사용 예시
vocab_size = 1000
embedding_dim = 50
hidden_size = 128

model = RNNWithHiddenTracking(vocab_size, embedding_dim, hidden_size)

# 예시 문장: "나는 점심 식사로 파스타를 먹을 예정입니다"
# (실제로는 토큰 ID로 변환된 값)
sentence = torch.tensor([[10, 25, 43, 87, 102, 156]])  # shape: (1, 6)
words = ["나는", "점심", "식사로", "파스타를", "먹을", "예정입니다"]

# Forward pass
all_hiddens, final_hidden = model(sentence)

print("=" * 70)
print("Hidden State의 변화 추적")
print("=" * 70)

# 각 hidden state의 통계 비교
for t in range(len(words)):
    h_t = all_hiddens[0, t]  # t번째 hidden state

    print(f"\n[시점 {t+1}] '{words[t]}' 처리 후 - h{t+1}")
    print(f"  크기: {h_t.shape}")
    print(f"  평균값: {h_t.mean().item():.6f}")
    print(f"  최댓값: {h_t.max().item():.6f}")
    print(f"  최솟값: {h_t.min().item():.6f}")

    # 이전 hidden state와의 변화량 계산
    if t > 0:
        h_prev = all_hiddens[0, t-1]
        change = torch.abs(h_t - h_prev).mean().item()
        print(f"  이전 상태와의 변화량: {change:.6f}")
        print(f"    → 새로운 단어 '{words[t]}'의 정보가 추가됨!")

print("\n" + "=" * 70)
print("핵심 포인트:")
print("=" * 70)
print("✓ h1은 '나는'만 보고 만들어진 벡터")
print("✓ h2는 h1 + '점심' 정보를 결합한 벡터")
print("✓ h3은 h2 + '식사로' 정보를 결합한 벡터")
print("✓ ...")
print("✓ h6은 전체 문장의 의미를 담은 최종 벡터")
print("\n하지만 RNN의 문제: h6에서 '나는'의 정보가 많이 희미해짐! (기울기 소실)")

Hidden State의 변화 추적

[시점 1] '나는' 처리 후 - h1
  크기: torch.Size([128])
  평균값: -0.008030
  최댓값: 0.622678
  최솟값: -0.781650

[시점 2] '점심' 처리 후 - h2
  크기: torch.Size([128])
  평균값: -0.021931
  최댓값: 0.746920
  최솟값: -0.701562
  이전 상태와의 변화량: 0.320961
    → 새로운 단어 '점심'의 정보가 추가됨!

[시점 3] '식사로' 처리 후 - h3
  크기: torch.Size([128])
  평균값: -0.008400
  최댓값: 0.688216
  최솟값: -0.808804
  이전 상태와의 변화량: 0.306977
    → 새로운 단어 '식사로'의 정보가 추가됨!

[시점 4] '파스타를' 처리 후 - h4
  크기: torch.Size([128])
  평균값: -0.015400
  최댓값: 0.731057
  최솟값: -0.824842
  이전 상태와의 변화량: 0.324786
    → 새로운 단어 '파스타를'의 정보가 추가됨!

[시점 5] '먹을' 처리 후 - h5
  크기: torch.Size([128])
  평균값: -0.024338
  최댓값: 0.724042
  최솟값: -0.778063
  이전 상태와의 변화량: 0.379232
    → 새로운 단어 '먹을'의 정보가 추가됨!

[시점 6] '예정입니다' 처리 후 - h6
  크기: torch.Size([128])
  평균값: 0.009061
  최댓값: 0.846417
  최솟값: -0.811501
  이전 상태와의 변화량: 0.442882
    → 새로운 단어 '예정입니다'의 정보가 추가됨!

핵심 포인트:
✓ h1은 '나는'만 보고 만들어진 벡터
✓ h2는 h1 + '점심' 정보를 결합한 벡터
✓ h3은 h2 + '식사로' 정보를 결합한 벡터
✓ ...
✓ h6은 전체 문장의 의미를 담은 최종 벡터

하지만 RNN

### 예제 3: Hidden State 간 유사도 비교

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def cosine_similarity_matrix(hidden_states):
    """각 hidden state 간의 코사인 유사도 계산"""
    # hidden_states: (seq_len, hidden_size)

    # 정규화
    normalized = F.normalize(hidden_states, p=2, dim=1)

    # 유사도 행렬 계산
    similarity = torch.mm(normalized, normalized.t())

    return similarity

# 모델 생성 및 실행
model = RNNWithHiddenTracking(vocab_size=1000, embedding_dim=50, hidden_size=128)
sentence = torch.tensor([[10, 25, 43, 87, 102, 156]])
all_hiddens, _ = model(sentence)

# Hidden states 추출 (batch 차원 제거)
hidden_states = all_hiddens.squeeze(0)  # (6, 128)

# 유사도 행렬 계산
similarity_matrix = cosine_similarity_matrix(hidden_states)

print("=" * 70)
print("Hidden State 간 유사도 분석")
print("=" * 70)
print("\n코사인 유사도 행렬:")
print("(1.0 = 완전히 같음, 0.0 = 완전히 다름)\n")

words = ["나는", "점심", "식사로", "파스타를", "먹을", "예정입니다"]

# 헤더 출력
print("        ", end="")
for w in words:
    print(f"{w:>8}", end="")
print()

# 행렬 출력
for i, word in enumerate(words):
    print(f"{word:>6}  ", end="")
    for j in range(len(words)):
        print(f"{similarity_matrix[i, j].item():>8.4f}", end="")
    print()

print("\n" + "=" * 70)
print("분석:")
print("=" * 70)
print("✓ 대각선 값들은 1.0 (자기 자신과의 유사도)")
print("✓ h1과 h6의 유사도가 낮음 → '나는' 정보가 h6에 적게 남음")
print("✓ 인접한 hidden state들의 유사도가 높음 → 점진적 변화")
print("✓ 이것이 RNN의 장기 의존성(long-term dependency) 문제!")

Hidden State 간 유사도 분석

코사인 유사도 행렬:
(1.0 = 완전히 같음, 0.0 = 완전히 다름)

              나는      점심     식사로    파스타를      먹을   예정입니다
    나는    1.0000 -0.1717 -0.1588 -0.0879  0.1138  0.1544
    점심   -0.1717  1.0000  0.1461  0.0233 -0.1443 -0.1860
   식사로   -0.1588  0.1461  1.0000  0.1548 -0.1451 -0.1732
  파스타를   -0.0879  0.0233  0.1548  1.0000  0.1298 -0.1235
    먹을    0.1138 -0.1443 -0.1451  0.1298  1.0000  0.2857
 예정입니다    0.1544 -0.1860 -0.1732 -0.1235  0.2857  1.0000

분석:
✓ 대각선 값들은 1.0 (자기 자신과의 유사도)
✓ h1과 h6의 유사도가 낮음 → '나는' 정보가 h6에 적게 남음
✓ 인접한 hidden state들의 유사도가 높음 → 점진적 변화
✓ 이것이 RNN의 장기 의존성(long-term dependency) 문제!




---



## **2. Seq2Seq with Attention(간소화 버전)**
- Seq2Seq + Attention**

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    """Seq2Seq의 Encoder 부분"""

    def __init__(self, input_vocab_size, embedding_dim, hidden_size):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)

    def forward(self, x):
        # x: (batch_size, seq_len)
        embedded = self.embedding(x)  # (batch_size, seq_len, embedding_dim)

        # 모든 시점의 hidden state 반환 (Attention을 위해 필요)
        outputs, (hidden, cell) = self.lstm(embedded)
        # outputs: (batch_size, seq_len, hidden_size) - 모든 시점의 hidden
        # hidden: (1, batch_size, hidden_size) - 마지막 hidden

        return outputs, hidden, cell


class AttentionDecoder(nn.Module):
    """Attention 메커니즘을 사용하는 Decoder"""

    def __init__(self, output_vocab_size, embedding_dim, hidden_size):
        super(AttentionDecoder, self).__init__()
        self.embedding = nn.Embedding(output_vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim + hidden_size, hidden_size, batch_first=True)

        # Attention 계산을 위한 레이어
        self.attention = nn.Linear(hidden_size * 2, 1)
        self.fc = nn.Linear(hidden_size, output_vocab_size)

    def forward(self, x, encoder_outputs, hidden, cell):
        # x: (batch_size, 1) - 현재 생성할 단어
        # encoder_outputs: (batch_size, src_len, hidden_size)

        embedded = self.embedding(x)  # (batch_size, 1, embedding_dim)

        # Attention 가중치 계산
        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]

        # hidden을 src_len만큼 복사
        hidden_repeated = hidden.permute(1, 0, 2).repeat(1, src_len, 1)
        # (batch_size, src_len, hidden_size)

        # Encoder의 각 hidden state와 현재 decoder hidden을 결합
        energy = torch.cat([hidden_repeated, encoder_outputs], dim=2)
        # (batch_size, src_len, hidden_size*2)

        # Attention score 계산
        attention_scores = self.attention(energy).squeeze(2)
        # (batch_size, src_len)

        # Softmax로 가중치 변환
        attention_weights = F.softmax(attention_scores, dim=1)
        # (batch_size, src_len)

        # Context vector 계산 (가중합)
        context = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs)
        # (batch_size, 1, hidden_size)

        # Context와 embedding을 결합하여 LSTM 입력
        lstm_input = torch.cat([embedded, context], dim=2)
        # (batch_size, 1, embedding_dim + hidden_size)

        output, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))

        # 최종 단어 예측
        prediction = self.fc(output.squeeze(1))
        # (batch_size, output_vocab_size)

        return prediction, hidden, cell, attention_weights


class Seq2SeqWithAttention(nn.Module):
    """Attention을 사용하는 완전한 Seq2Seq 모델"""

    def __init__(self, input_vocab_size, output_vocab_size, embedding_dim, hidden_size):
        super(Seq2SeqWithAttention, self).__init__()
        self.encoder = Encoder(input_vocab_size, embedding_dim, hidden_size)
        self.decoder = AttentionDecoder(output_vocab_size, embedding_dim, hidden_size)

    def forward(self, src, trg):
        # src: (batch_size, src_len) - "나는 점심 식사로..."
        # trg: (batch_size, trg_len) - "I am going to..."

        # Encoding
        encoder_outputs, hidden, cell = self.encoder(src)

        # Decoding with Attention
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.fc.out_features

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size)
        attention_weights_all = []

        # 첫 입력은 <START> 토큰
        decoder_input = trg[:, 0].unsqueeze(1)

        for t in range(1, trg_len):
            # 한 단어씩 생성
            output, hidden, cell, attention_weights = self.decoder(
                decoder_input, encoder_outputs, hidden, cell
            )

            outputs[:, t] = output
            attention_weights_all.append(attention_weights)

            # 다음 입력 (teacher forcing 사용)
            decoder_input = trg[:, t].unsqueeze(1)

        return outputs, attention_weights_all


# 사용 예시
model = Seq2SeqWithAttention(
    input_vocab_size=1000,
    output_vocab_size=1000,
    embedding_dim=256,
    hidden_size=512
)

# 예시 데이터
src = torch.tensor([[10, 25, 43, 87, 102, 156]])  # "나는 점심 식사로 파스타를 먹을 예정입니다"
trg = torch.tensor([[1, 5, 8, 12, 15, 20, 30]])   # "<START> I am going to eat pasta"

outputs, attention_weights = model(src, trg)

print(f"Seq2Seq 출력 shape: {outputs.shape}")
print(f"Attention 가중치 개수: {len(attention_weights)}")
print("\n'eat' 생성 시 Attention 분포 (예시):")
print("나는(0.02), 점심(0.05), 식사로(0.1), 파스타를(0.6), 먹을(0.2), 예정입니다(0.03)")
print("→ '파스타를'과 '먹을'에 집중!")

Seq2Seq 출력 shape: torch.Size([1, 7, 1000])
Attention 가중치 개수: 6

'eat' 생성 시 Attention 분포 (예시):
나는(0.02), 점심(0.05), 식사로(0.1), 파스타를(0.6), 먹을(0.2), 예정입니다(0.03)
→ '파스타를'과 '먹을'에 집중!


## **3. Transformer구현(간소화 버전)**

In [6]:
import torch
import torch.nn as nn
import math

class MultiHeadAttention(nn.Module):
    """Transformer의 핵심: Multi-Head Self-Attention"""

    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        # Query, Key, Value 변환을 위한 linear layer
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Q, K, V: (batch_size, num_heads, seq_len, d_k)

        # Attention score 계산
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        # (batch_size, num_heads, seq_len, seq_len)

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        # Softmax로 가중치 계산
        attention_weights = torch.softmax(scores, dim=-1)

        # Value에 가중치 적용
        output = torch.matmul(attention_weights, V)

        return output, attention_weights

    def forward(self, Q, K, V, mask=None):
        batch_size = Q.shape[0]

        # Linear transformation
        Q = self.W_q(Q)  # (batch_size, seq_len, d_model)
        K = self.W_k(K)
        V = self.W_v(V)

        # Multi-head로 분할
        Q = Q.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = K.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = V.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        # (batch_size, num_heads, seq_len, d_k)

        # Attention 계산
        attn_output, attention_weights = self.scaled_dot_product_attention(Q, K, V, mask)

        # Head들을 다시 결합
        attn_output = attn_output.transpose(1, 2).contiguous()
        attn_output = attn_output.view(batch_size, -1, self.d_model)

        # 최종 linear transformation
        output = self.W_o(attn_output)

        return output, attention_weights


class PositionalEncoding(nn.Module):
    """단어의 위치 정보를 인코딩 (RNN 없이 순서 보존)"""

    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()

        # 위치 인코딩 계산
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                            (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        seq_len = x.shape[1]
        return x + self.pe[:, :seq_len, :]


class TransformerEncoderLayer(nn.Module):
    """Transformer Encoder의 한 레이어"""

    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(TransformerEncoderLayer, self).__init__()

        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # Self-Attention (모든 단어가 서로 관계 파악)
        attn_output, attn_weights = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))  # Residual connection

        # Feed Forward
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))  # Residual connection

        return x, attn_weights


class SimpleTransformer(nn.Module):
    """간소화된 Transformer 번역 모델"""

    def __init__(self, src_vocab_size, trg_vocab_size, d_model=512,
                 num_heads=8, num_layers=6, d_ff=2048, dropout=0.1):
        super(SimpleTransformer, self).__init__()

        # Embedding layers
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.trg_embedding = nn.Embedding(trg_vocab_size, d_model)

        # Positional Encoding
        self.pos_encoding = PositionalEncoding(d_model)

        # Encoder layers
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])

        # Decoder layers (간소화 - 실제로는 더 복잡함)
        self.decoder_layers = nn.ModuleList([
            TransformerEncoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])

        # Output layer
        self.fc_out = nn.Linear(d_model, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.d_model = d_model

    def encode(self, src):
        # src: (batch_size, src_len)

        # Embedding + Positional Encoding
        x = self.src_embedding(src) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)

        # 여러 Encoder 레이어 통과 (병렬 처리!)
        attention_weights_list = []
        for layer in self.encoder_layers:
            x, attn_weights = layer(x)
            attention_weights_list.append(attn_weights)

        return x, attention_weights_list

    def decode(self, trg, encoder_output):
        # trg: (batch_size, trg_len)

        # Embedding + Positional Encoding
        x = self.trg_embedding(trg) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)

        # Decoder 레이어 통과
        for layer in self.decoder_layers:
            x, _ = layer(x)

        # 최종 출력
        output = self.fc_out(x)

        return output

    def forward(self, src, trg):
        # Encoding (전체 문장을 한 번에 병렬 처리!)
        encoder_output, attention_weights = self.encode(src)

        # Decoding
        output = self.decode(trg, encoder_output)

        return output, attention_weights


# ============================================================
# 사용 예시 및 테스트
# ============================================================

print("=" * 80)
print("Transformer 모델 생성 및 테스트")
print("=" * 80)

# Transformer 모델 생성
transformer = SimpleTransformer(
    src_vocab_size=1000,
    trg_vocab_size=1000,
    d_model=512,
    num_heads=8,
    num_layers=6
)

# 모델 파라미터 수 계산
total_params = sum(p.numel() for p in transformer.parameters())
trainable_params = sum(p.numel() for p in transformer.parameters() if p.requires_grad)

print(f"\n모델 정보:")
print(f"  총 파라미터 수: {total_params:,}")
print(f"  학습 가능한 파라미터 수: {trainable_params:,}")

# 예시 데이터
src = torch.tensor([[10, 25, 43, 87, 102, 156]])  # "나는 점심 식사로 파스타를 먹을 예정입니다"
trg = torch.tensor([[1, 5, 8, 12, 15, 20]])       # "<START> I am going to eat"

print(f"\n입력 데이터:")
print(f"  소스 문장 shape: {src.shape}")
print(f"  타겟 문장 shape: {trg.shape}")

# Forward pass
output, attention_weights = transformer(src, trg)

print(f"\n출력 결과:")
print(f"  Transformer 출력 shape: {output.shape}")
print(f"  (batch_size=1, sequence_length=6, vocab_size=1000)")
print(f"\n  Attention 레이어 수: {len(attention_weights)}")
print(f"  각 레이어의 Attention shape: {attention_weights[0].shape}")
print(f"  (batch_size=1, num_heads=8, seq_len=6, seq_len=6)")

print("\n" + "=" * 80)
print("Transformer의 핵심 특징")
print("=" * 80)
print("1. 모든 단어가 동시에 처리됨 (병렬 처리)")
print("2. Self-Attention으로 '나는'과 '먹을'의 관계를 직접 파악")
print("3. Positional Encoding으로 순서 정보 보존")
print("4. RNN보다 훨씬 빠르고 긴 문장도 효과적으로 처리 가능")
print("5. 현재 GPT, BERT, T5 등 모든 최신 LLM의 기반 아키텍처")

# ============================================================
# Self-Attention 시각화 예제
# ============================================================

print("\n" + "=" * 80)
print("Self-Attention 가중치 분석")
print("=" * 80)

# 첫 번째 레이어의 첫 번째 헤드의 attention 가중치
first_layer_attention = attention_weights[0][0, 0]  # (seq_len, seq_len)

words = ["나는", "점심", "식사로", "파스타를", "먹을", "예정입니다"]

print("\n첫 번째 Encoder 레이어, 첫 번째 헤드의 Attention 가중치:")
print("(각 행은 해당 단어가 다른 단어들에 주는 attention)\n")

# 헤더 출력
print("          ", end="")
for w in words:
    print(f"{w:>8}", end="")
print("\n" + "-" * 70)

# Attention 행렬 출력
for i, word in enumerate(words):
    print(f"{word:>8}  ", end="")
    for j in range(len(words)):
        attention_score = first_layer_attention[i, j].item()
        print(f"{attention_score:>8.4f}", end="")
    print()

print("\n분석:")
print("  ✓ 대각선 값이 높음: 자기 자신에게 attention")
print("  ✓ '나는'과 '먹을' 간 높은 값: 주어-동사 관계")
print("  ✓ '파스타를'과 '먹을' 간 높은 값: 목적어-동사 관계")
print("  ✓ RNN과 달리 거리에 관계없이 직접 연결!")

# ============================================================
# 처리 속도 비교 (개념적 설명)
# ============================================================

print("\n" + "=" * 80)
print("RNN vs Transformer 처리 방식 비교")
print("=" * 80)

print("\n[RNN의 처리 방식 - 순차적]")
print("  시간 t=1: '나는' 처리      → h1 생성")
print("  시간 t=2: h1 + '점심' 처리 → h2 생성")
print("  시간 t=3: h2 + '식사로' 처리 → h3 생성")
print("  시간 t=4: h3 + '파스타를' 처리 → h4 생성")
print("  시간 t=5: h4 + '먹을' 처리 → h5 생성")
print("  시간 t=6: h5 + '예정입니다' 처리 → h6 생성")
print("  → 총 6단계 필요 (순차적, 병렬화 불가)")

print("\n[Transformer의 처리 방식 - 병렬적]")
print("  시간 t=1: 모든 단어를 동시에 처리!")
print("            '나는', '점심', '식사로', '파스타를', '먹을', '예정입니다'")
print("            → 모든 위치의 representation 동시 생성")
print("  → 총 1단계 필요 (병렬 처리 가능, GPU 효율 극대화)")

print("\n성능 차이:")
print("  • RNN: 문장 길이 n에 비례하여 처리 시간 증가 O(n)")
print("  • Transformer: 문장 길이와 무관하게 일정 시간 O(1) (병렬 처리 시)")
print("  • 실제 학습 속도: Transformer가 RNN보다 10-100배 빠름")

# ============================================================
# 장거리 의존성 처리 비교
# ============================================================

print("\n" + "=" * 80)
print("장거리 의존성(Long-term Dependency) 처리 능력")
print("=" * 80)

print("\n예시 문장: '나는 어제 친구와 영화관에 가서 재미있는 영화를 봤다'")
print("          (거리: '나는' ←→ '봤다' 사이에 7개 단어)")

print("\n[RNN]")
print("  '나는' → h1 → h2 → h3 → h4 → h5 → h6 → h7 → h8 → '봤다'")
print("  문제: 8단계를 거치면서 '나는'의 정보가 희미해짐 (기울기 소실)")
print("  결과: '나는'과 '봤다'의 주어-동사 관계를 제대로 파악하기 어려움")

print("\n[Transformer]")
print("  '나는' ←─────────────────────────────────→ '봤다'")
print("                (Self-Attention으로 직접 연결)")
print("  장점: 거리에 관계없이 직접 attention 가능")
print("  결과: '나는'과 '봤다'의 관계를 정확히 파악!")

# ============================================================
# 실제 응용 사례
# ============================================================

print("\n" + "=" * 80)
print("Transformer 기반 최신 AI 모델들")
print("=" * 80)

models_info = [
    ("BERT", "2018", "Google", "양방향 문맥 이해, 문장 분류/QA"),
    ("GPT-2", "2019", "OpenAI", "텍스트 생성, 대화"),
    ("GPT-3", "2020", "OpenAI", "Few-shot learning, 범용 AI"),
    ("T5", "2019", "Google", "Text-to-Text 통합 프레임워크"),
    ("GPT-4", "2023", "OpenAI", "멀티모달, 추론 능력 향상"),
    ("Claude", "2023", "Anthropic", "안전하고 유용한 AI 어시스턴트"),
]

print("\n모델명    연도    기관        특징")
print("-" * 80)
for name, year, org, feature in models_info:
    print(f"{name:8} {year}   {org:10} {feature}")

print("\n→ 모든 모델이 Transformer 아키텍처를 기반으로 함!")
print("→ 'Attention is All You Need' 논문(2017)이 AI 혁명의 시작점")

print("\n" + "=" * 80)
print("요약: Transformer가 RNN을 대체한 이유")
print("=" * 80)
print("1. ✓ 병렬 처리 → 학습 속도 10-100배 빠름")
print("2. ✓ 장거리 의존성 → 긴 문장도 정확히 이해")
print("3. ✓ 확장성 → 모델 크기를 키울수록 성능 향상")
print("4. ✓ 범용성 → 번역, 분류, 생성 등 모든 NLP 작업에 적용 가능")
print("5. ✓ 해석 가능성 → Attention 가중치로 모델 동작 이해 가능")
print("=" * 80)

Transformer 모델 생성 및 테스트

모델 정보:
  총 파라미터 수: 39,365,608
  학습 가능한 파라미터 수: 39,365,608

입력 데이터:
  소스 문장 shape: torch.Size([1, 6])
  타겟 문장 shape: torch.Size([1, 6])

출력 결과:
  Transformer 출력 shape: torch.Size([1, 6, 1000])
  (batch_size=1, sequence_length=6, vocab_size=1000)

  Attention 레이어 수: 6
  각 레이어의 Attention shape: torch.Size([1, 8, 6, 6])
  (batch_size=1, num_heads=8, seq_len=6, seq_len=6)

Transformer의 핵심 특징
1. 모든 단어가 동시에 처리됨 (병렬 처리)
2. Self-Attention으로 '나는'과 '먹을'의 관계를 직접 파악
3. Positional Encoding으로 순서 정보 보존
4. RNN보다 훨씬 빠르고 긴 문장도 효과적으로 처리 가능
5. 현재 GPT, BERT, T5 등 모든 최신 LLM의 기반 아키텍처

Self-Attention 가중치 분석

첫 번째 Encoder 레이어, 첫 번째 헤드의 Attention 가중치:
(각 행은 해당 단어가 다른 단어들에 주는 attention)

                나는      점심     식사로    파스타를      먹을   예정입니다
----------------------------------------------------------------------
      나는    0.0000  0.0000  1.0000  0.0000  0.0000  0.0000
      점심    0.0000  0.0000  1.0000  0.0000  0.0000  0.0000
     식사로    0.0000  1.0000  0.0000  0.0000  0.0000  0.0000